In [12]:
from langchain.globals import set_debug
set_debug(True)

## 문자열 템플릿

In [13]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("안녕하세요, 제 이름은 {name}이고, 나이는 {age}살입니다.")

# 템플릿에 값을 채워서 프롬프트를 완성
filled_prompt = prompt_template.format(name="홍길동", age=30)
filled_prompt


'안녕하세요, 제 이름은 홍길동이고, 나이는 30살입니다.'

### PromptTemplate 결합

In [14]:
prompt1 = PromptTemplate.from_template("안녕하세요, 제 이름은 {name}이고, 나이는 {age}살입니다.")
prompt2 = PromptTemplate.from_template("\n\n아버지를 아버지라 부를 수 없습니다.")
prompt3 = "\n\n{language}로 번역해주세요."

combined_prompt = (
    prompt1
    + prompt2
    + prompt3
)
combined_prompt

PromptTemplate(input_variables=['age', 'language', 'name'], template='안녕하세요, 제 이름은 {name}이고, 나이는 {age}살입니다.\n\n아버지를 아버지라 부를 수 없습니다.\n\n{language}로 번역해주세요.')

In [15]:
combined_prompt.format(name="홍길동", age=30, language="영어")

'안녕하세요, 제 이름은 홍길동이고, 나이는 30살입니다.\n\n아버지를 아버지라 부를 수 없습니다.\n\n영어로 번역해주세요.'

In [16]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers.string import StrOutputParser

llm = ChatGoogleGenerativeAI(model="gemini-pro")
chain = combined_prompt | llm | StrOutputParser()
chain.invoke({"name": "홍길동", "age": 30, "language": "영어"})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "name": "홍길동",
  "age": 30,
  "language": "영어"
}
[chain/start] [chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{
  "name": "홍길동",
  "age": 30,
  "language": "영어"
}
[chain/end] [chain:RunnableSequence > prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatGoogleGenerativeAI] Entering LLM run with input:
{
  "prompts": [
    "Human: 안녕하세요, 제 이름은 홍길동이고, 나이는 30살입니다.\n\n아버지를 아버지라 부를 수 없습니다.\n\n영어로 번역해주세요."
  ]
}
[llm/end] [chain:RunnableSequence > llm:ChatGoogleGenerativeAI] [1.46s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Hello, my name is Hong Gildong, and I am 30 years old.\n\nI cannot call my father, father.",
        "generation_info": {
          "finish_reason": "STOP",
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      

'Hello, my name is Hong Gildong, and I am 30 years old.\n\nI cannot call my father, father.'

## ChatPromptTemplate

- SystemMessage: 시스템의 기능을 설명합니다.
- HumanMessage: 사용자의 질문을 나타냅니다.
- AIMessage: AI 모델의 응답을 제공합니다.
- FunctionMessage: 특정 함수 호출의 결과를 나타냅니다.
- ToolMessage: 도구 호출의 결과를 나타냅니다.

### tuple 형태의 메세지 리스트

In [17]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "이 시스템은 천문학 질문에 답변할 수 있습니다."), # (type, content) tuple
    ("user", "{user_input}"), # (type, content) tuple
])

chat_prompt.format_messages(user_input="태양계에서 가장 큰 행성은 무엇인가요?")

[SystemMessage(content='이 시스템은 천문학 질문에 답변할 수 있습니다.'),
 HumanMessage(content='태양계에서 가장 큰 행성은 무엇인가요?')]

In [18]:
from langchain_core.output_parsers import StrOutputParser

llm = ChatGoogleGenerativeAI(model="gemini-pro")
chain = chat_prompt | llm | StrOutputParser()

chain.invoke({"user_input": "태양계에서 가장 큰 행성은 무엇인가요?"})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "user_input": "태양계에서 가장 큰 행성은 무엇인가요?"
}
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "user_input": "태양계에서 가장 큰 행성은 무엇인가요?"
}
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatGoogleGenerativeAI] Entering LLM run with input:
{
  "prompts": [
    "System: 이 시스템은 천문학 질문에 답변할 수 있습니다.\nHuman: 태양계에서 가장 큰 행성은 무엇인가요?"
  ]
}
[llm/end] [chain:RunnableSequence > llm:ChatGoogleGenerativeAI] [1.80s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "목성",
        "generation_info": {
          "finish_reason": "STOP",
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE",
              "blocked": false
            },
            {
              

'목성'

### MessagePromptTemplate 활용

In [20]:
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate

chat_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template("이 시스템은 천문학 질문에 답변할 수 있습니다."),
        HumanMessagePromptTemplate.from_template("{user_input}"),
    ]
)
chat_prompt

ChatPromptTemplate(input_variables=['user_input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='이 시스템은 천문학 질문에 답변할 수 있습니다.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_input'], template='{user_input}'))])

In [21]:
chat_prompt.format_messages(user_input="태양계에서 가장 큰 행성은 무엇인가요?")

[SystemMessage(content='이 시스템은 천문학 질문에 답변할 수 있습니다.'),
 HumanMessage(content='태양계에서 가장 큰 행성은 무엇인가요?')]

In [24]:
llm = ChatGoogleGenerativeAI(model="gemini-pro")
chain = chat_prompt | llm | StrOutputParser()

chain.invoke({"user_input": "태양계에서 가장 큰 행성은 무엇인가요?"})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "user_input": "태양계에서 가장 큰 행성은 무엇인가요?"
}
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "user_input": "태양계에서 가장 큰 행성은 무엇인가요?"
}
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatGoogleGenerativeAI] Entering LLM run with input:
{
  "prompts": [
    "System: 이 시스템은 천문학 질문에 답변할 수 있습니다.\nHuman: 태양계에서 가장 큰 행성은 무엇인가요?"
  ]
}
[llm/end] [chain:RunnableSequence > llm:ChatGoogleGenerativeAI] [2.33s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "목성(Jupiter)",
        "generation_info": {
          "finish_reason": "STOP",
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE",
              "blocked": false
            },
            {
     

'목성(Jupiter)'